# شبکه‌های مولد

شبکه‌های عصبی بازگشتی (RNNs) و انواع سلول‌های دروازه‌دار آن‌ها مانند سلول‌های حافظه کوتاه‌مدت بلند (LSTMs) و واحدهای بازگشتی دروازه‌دار (GRUs) مکانیزمی برای مدل‌سازی زبان فراهم کردند، یعنی می‌توانند ترتیب کلمات را یاد بگیرند و پیش‌بینی‌هایی برای کلمه بعدی در یک دنباله ارائه دهند. این قابلیت به ما اجازه می‌دهد از RNNها برای **وظایف مولد** استفاده کنیم، مانند تولید متن معمولی، ترجمه ماشینی، و حتی توضیح‌نویسی تصاویر.

در معماری RNN که در واحد قبلی بحث کردیم، هر واحد RNN حالت مخفی بعدی را به عنوان خروجی تولید می‌کرد. با این حال، می‌توانیم یک خروجی دیگر به هر واحد بازگشتی اضافه کنیم که به ما اجازه می‌دهد یک **دنباله** خروجی تولید کنیم (که طول آن برابر با دنباله اصلی است). علاوه بر این، می‌توانیم از واحدهای RNN استفاده کنیم که در هر مرحله ورودی دریافت نمی‌کنند و فقط یک بردار حالت اولیه می‌گیرند و سپس یک دنباله خروجی تولید می‌کنند.

در این دفترچه، ما بر مدل‌های مولد ساده تمرکز خواهیم کرد که به ما کمک می‌کنند متن تولید کنیم. برای ساده‌سازی، بیایید یک **شبکه سطح کاراکتر** بسازیم که متن را حرف به حرف تولید کند. در طول آموزش، باید یک متن مرجع را بگیریم و آن را به دنباله‌های حرف تقسیم کنیم.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## ساخت واژگان کاراکتری

برای ساخت یک شبکه مولد در سطح کاراکتر، باید متن را به کاراکترهای جداگانه تقسیم کنیم، نه کلمات. لایه `TextVectorization` که قبلاً استفاده می‌کردیم نمی‌تواند این کار را انجام دهد، بنابراین دو گزینه داریم:

* متن را به صورت دستی بارگذاری کرده و توکن‌سازی را «دستی» انجام دهیم، همان‌طور که در [این مثال رسمی Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) آمده است.
* از کلاس `Tokenizer` برای توکن‌سازی در سطح کاراکتر استفاده کنیم.

ما گزینه دوم را انتخاب می‌کنیم. `Tokenizer` همچنین می‌تواند برای توکن‌سازی در سطح کلمه استفاده شود، بنابراین می‌توان به راحتی از توکن‌سازی کاراکتری به توکن‌سازی کلمه‌ای تغییر حالت داد.

برای انجام توکن‌سازی در سطح کاراکتر، باید پارامتر `char_level=True` را ارسال کنیم:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

ما همچنین می‌خواهیم از یک نشانه ویژه برای نشان دادن **پایان دنباله** استفاده کنیم، که آن را `<eos>` می‌نامیم. بیایید آن را به صورت دستی به واژگان اضافه کنیم:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

اکنون، برای رمزگذاری متن به دنباله‌های اعداد، می‌توانیم استفاده کنیم:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## آموزش یک RNN مولد برای تولید عناوین

روش آموزش RNN برای تولید عناوین خبری به این صورت است: در هر مرحله، یک عنوان را می‌گیریم، آن را به RNN می‌دهیم و از شبکه می‌خواهیم برای هر کاراکتر ورودی، کاراکتر خروجی بعدی را تولید کند:

![تصویری که نمونه‌ای از تولید RNN برای کلمه 'HELLO' را نشان می‌دهد.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

برای آخرین کاراکتر از دنباله‌مان، از شبکه می‌خواهیم که توکن `<eos>` را تولید کند.

تفاوت اصلی RNN مولدی که اینجا استفاده می‌کنیم این است که خروجی هر مرحله از RNN را می‌گیریم، نه فقط خروجی سلول نهایی. این کار با مشخص کردن پارامتر `return_sequences` برای سلول RNN امکان‌پذیر است.

بنابراین، در طول آموزش، ورودی شبکه یک دنباله از کاراکترهای کدگذاری‌شده با طول مشخص خواهد بود و خروجی نیز دنباله‌ای با همان طول خواهد بود که یک عنصر جابه‌جا شده و با `<eos>` خاتمه می‌یابد. مینی‌بچ شامل چندین دنباله از این نوع خواهد بود و برای هم‌تراز کردن تمام دنباله‌ها نیاز به استفاده از **پدینگ** داریم.

بیایید توابعی ایجاد کنیم که مجموعه داده را برای ما تبدیل کنند. از آنجا که می‌خواهیم دنباله‌ها را در سطح مینی‌بچ پد کنیم، ابتدا مجموعه داده را با فراخوانی `.batch()` به صورت بچ تقسیم می‌کنیم و سپس با استفاده از `map` آن را برای انجام تبدیل تغییر می‌دهیم. بنابراین، تابع تبدیل کل مینی‌بچ را به عنوان یک پارامتر دریافت خواهد کرد:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

چند نکته مهم که در اینجا انجام می‌دهیم:
* ابتدا متن واقعی را از رشته‌ی تنسور استخراج می‌کنیم
* `text_to_sequences` لیستی از رشته‌ها را به لیستی از تنسورهای عددی تبدیل می‌کند
* `pad_sequences` این تنسورها را تا طول حداکثرشان پر می‌کند
* در نهایت تمام کاراکترها را به صورت یک‌هاتی کدگذاری می‌کنیم، و همچنین جابجایی و افزودن `<eos>` را انجام می‌دهیم. به زودی خواهیم دید چرا به کاراکترهای یک‌هاتی کدگذاری‌شده نیاز داریم

با این حال، این تابع **پایتون‌محور** است، یعنی نمی‌توان آن را به طور خودکار به گراف محاسباتی Tensorflow ترجمه کرد. اگر بخواهیم این تابع را مستقیماً در تابع `Dataset.map` استفاده کنیم، با خطا مواجه خواهیم شد. باید این فراخوانی پایتون‌محور را با استفاده از پوشش‌دهنده‌ی `py_function` محصور کنیم:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **توجه**: ممکن است تشخیص تفاوت بین توابع تبدیل پایتونی و توابع تبدیل Tensorflow کمی پیچیده به نظر برسد و شاید این سوال برایتان پیش بیاید که چرا داده‌ها را با استفاده از توابع استاندارد پایتون قبل از ارسال به `fit` تبدیل نمی‌کنیم. در حالی که این کار قطعاً امکان‌پذیر است، استفاده از `Dataset.map` یک مزیت بزرگ دارد، زیرا خط لوله تبدیل داده‌ها با استفاده از گراف محاسباتی Tensorflow اجرا می‌شود که از محاسبات GPU بهره می‌برد و نیاز به انتقال داده‌ها بین CPU و GPU را به حداقل می‌رساند.

حالا می‌توانیم شبکه مولد خود را بسازیم و آموزش را شروع کنیم. این شبکه می‌تواند بر اساس هر سلول بازگشتی که در واحد قبلی بحث کردیم (ساده، LSTM یا GRU) باشد. در مثال ما از LSTM استفاده خواهیم کرد.

از آنجا که شبکه کاراکترها را به عنوان ورودی می‌گیرد و اندازه واژگان نسبتاً کوچک است، نیازی به لایه تعبیه (embedding) نداریم و ورودی‌های کدگذاری‌شده به صورت یک‌داغ (one-hot-encoded) می‌توانند مستقیماً وارد سلول LSTM شوند. لایه خروجی یک طبقه‌بند `Dense` خواهد بود که خروجی LSTM را به اعداد کدگذاری‌شده به صورت یک‌داغ تبدیل می‌کند.

علاوه بر این، از آنجا که با دنباله‌هایی با طول متغیر سروکار داریم، می‌توانیم از لایه `Masking` استفاده کنیم تا ماسکی ایجاد کنیم که بخش پرشده رشته را نادیده بگیرد. این کار به طور دقیق ضروری نیست، زیرا ما خیلی به آنچه که فراتر از توکن `<eos>` می‌رود علاقه‌مند نیستیم، اما برای کسب تجربه با این نوع لایه از آن استفاده خواهیم کرد. `input_shape` برابر با `(None, vocab_size)` خواهد بود، که در آن `None` نشان‌دهنده دنباله‌ای با طول متغیر است، و شکل خروجی نیز `(None, vocab_size)` خواهد بود، همانطور که می‌توانید از `summary` مشاهده کنید:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## تولید خروجی

حالا که مدل را آموزش داده‌ایم، می‌خواهیم از آن برای تولید خروجی استفاده کنیم. ابتدا، نیاز داریم روشی برای رمزگشایی متنی که به صورت دنباله‌ای از اعداد توکن نمایش داده شده است، داشته باشیم. برای این کار می‌توانیم از تابع `tokenizer.sequences_to_texts` استفاده کنیم؛ اما این تابع با توکن‌سازی در سطح کاراکتر به خوبی کار نمی‌کند. بنابراین، یک دیکشنری از توکن‌ها از `tokenizer` (به نام `word_index`) می‌گیریم، یک نقشه معکوس می‌سازیم و تابع رمزگشایی خودمان را می‌نویسیم:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

حالا، بیایید تولید را شروع کنیم. ابتدا یک رشته `start` داریم که آن را به یک دنباله `inp` رمزگذاری می‌کنیم، و سپس در هر مرحله شبکه خود را فراخوانی می‌کنیم تا کاراکتر بعدی را پیش‌بینی کند.

خروجی شبکه `out` یک بردار با `vocab_size` عنصر است که احتمال هر نشانه را نشان می‌دهد، و می‌توانیم با استفاده از `argmax` شماره نشانه‌ای که بیشترین احتمال را دارد پیدا کنیم. سپس این کاراکتر را به لیست نشانه‌های تولید شده اضافه می‌کنیم و فرآیند تولید را ادامه می‌دهیم. این فرآیند تولید یک کاراکتر، `size` بار تکرار می‌شود تا تعداد کاراکترهای مورد نیاز تولید شود، و در صورتی که `eos_token` زودتر مواجه شود، تولید متوقف می‌شود.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## نمونه‌گیری خروجی در حین آموزش

از آنجا که ما هیچ معیار مفیدی مانند *دقت* نداریم، تنها راهی که می‌توانیم ببینیم مدل ما بهتر می‌شود یا نه، **نمونه‌گیری** از رشته‌های تولیدشده در حین آموزش است. برای انجام این کار، از **callbacks** استفاده خواهیم کرد، یعنی توابعی که می‌توانیم به تابع `fit` بدهیم و این توابع به صورت دوره‌ای در طول آموزش فراخوانی می‌شوند.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


این مثال از قبل متن نسبتاً خوبی تولید می‌کند، اما می‌توان آن را به چندین روش بهبود داد:

* **متن بیشتر**. ما فقط از عناوین برای وظیفه خود استفاده کرده‌ایم، اما ممکن است بخواهید با متن کامل آزمایش کنید. به یاد داشته باشید که RNNها در مدیریت دنباله‌های طولانی چندان خوب نیستند، بنابراین منطقی است که یا آن‌ها را به جملات کوتاه‌تر تقسیم کنید، یا همیشه روی طول دنباله‌ای ثابت با مقداری از پیش تعریف‌شده `num_chars` (مثلاً ۲۵۶) آموزش دهید. می‌توانید مثال بالا را به چنین معماری‌ای تغییر دهید و از [آموزش رسمی Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) الهام بگیرید.

* **LSTM چندلایه**. منطقی است که ۲ یا ۳ لایه از سلول‌های LSTM را امتحان کنید. همان‌طور که در واحد قبلی اشاره کردیم، هر لایه از LSTM الگوهای خاصی را از متن استخراج می‌کند، و در مورد تولیدکننده در سطح کاراکتر، می‌توان انتظار داشت که لایه‌های پایین‌تر LSTM مسئول استخراج هجاها باشند و لایه‌های بالاتر - کلمات و ترکیب‌های کلمه‌ای. این کار به سادگی با ارسال پارامتر تعداد لایه‌ها به سازنده LSTM قابل پیاده‌سازی است.

* همچنین ممکن است بخواهید با **واحدهای GRU** آزمایش کنید و ببینید کدام‌یک عملکرد بهتری دارند، و با **اندازه‌های مختلف لایه‌های مخفی** نیز کار کنید. لایه مخفی بیش از حد بزرگ ممکن است منجر به بیش‌برازش شود (برای مثال، شبکه متن دقیق را یاد می‌گیرد)، و اندازه کوچک‌تر ممکن است نتیجه خوبی تولید نکند.


## تولید متن نرم و دما

در تعریف قبلی `generate`، همیشه کاراکتری که بالاترین احتمال را داشت به عنوان کاراکتر بعدی در متن تولید شده انتخاب می‌شد. این باعث می‌شد که متن اغلب بین دنباله‌های کاراکتری مشابه بارها و بارها "چرخش" کند، مانند این مثال:
```
today of the second the company and a second the company ...
```

با این حال، اگر به توزیع احتمالات برای کاراکتر بعدی نگاه کنیم، ممکن است تفاوت بین چند احتمال بالاتر خیلی زیاد نباشد، به عنوان مثال یک کاراکتر می‌تواند احتمال 0.2 داشته باشد و دیگری 0.19 و غیره. برای مثال، وقتی به دنبال کاراکتر بعدی در دنباله '*play*' هستیم، کاراکتر بعدی می‌تواند به همان اندازه فضای خالی باشد یا **e** (مانند کلمه *player*).

این ما را به این نتیجه می‌رساند که همیشه "منصفانه" نیست که کاراکتری با احتمال بالاتر را انتخاب کنیم، زیرا انتخاب کاراکتر دوم با احتمال بالاتر نیز ممکن است به متنی معنادار منجر شود. عاقلانه‌تر است که کاراکترها را از توزیع احتمالاتی که توسط خروجی شبکه داده شده است **نمونه‌گیری** کنیم.

این نمونه‌گیری می‌تواند با استفاده از تابع `np.multinomial` انجام شود که توزیع احتمالی **چندجمله‌ای** را پیاده‌سازی می‌کند. تابعی که این تولید متن **نرم** را پیاده‌سازی می‌کند در زیر تعریف شده است:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

ما یک پارامتر دیگر به نام **دما** معرفی کرده‌ایم که برای نشان دادن میزان پایبندی به بالاترین احتمال استفاده می‌شود. اگر دما ۱.۰ باشد، نمونه‌گیری چندجمله‌ای منصفانه انجام می‌دهیم، و زمانی که دما به بی‌نهایت می‌رسد - همه احتمالات برابر می‌شوند و ما به صورت تصادفی کاراکتر بعدی را انتخاب می‌کنیم. در مثال زیر می‌توان مشاهده کرد که متن با افزایش بیش از حد دما بی‌معنی می‌شود و زمانی که دما به ۰ نزدیک‌تر می‌شود، به متن سخت-تولید شده "چرخه‌ای" شباهت پیدا می‌کند.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
